# Import library
those are the requirement for the application

In [2]:
!pip install youtube-transcript-api google-generativeai googlesearch-python


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import os
import re
import json
import time
import requests
import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import Markdown, display
from youtube_transcript_api import YouTubeTranscriptApi
from googlesearch import search
from bs4 import BeautifulSoup



# Google search

In [32]:
class YoutubeSearch:
    def __init__(self, keyword, num_results=10, lang="en"):
        self.site = "www.youtube.com"
        self.keywork = keyword
        self.num_results = num_results
        self.lang = lang
        self.results = self.get_link()
        self.titles = self.get_titles()
        

    def get_link(self):
        results = [ result for result in search(f"site:{self.site} {self.keywork}", 
                                               num_results=self.num_results, 
                                              lang=self.lang)][1:]
        return results
        
    def get_titles(self):
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
        titles=[]
        for url in self.results:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.title.string if soup.title else "No title found"
            titles.append((title,url))
        return titles
        

In [33]:
ytLinks = YoutubeSearch(keyword="AI")
ytLinks.titles

[('Artificial Intelligence Full Course 2025 | Artificial Intelligence Tutorial | AI Course |Simplilearn - YouTube',
  'https://www.youtube.com/watch?v=vScGLqbeWbs'),
 ('AI is changing war. Just not with killer robots - YouTube',
  'https://www.youtube.com/watch?v=geaXM1EwZlg'),
 ('DeepSeek’s Lessons for Chinese AI - YouTube',
  'https://www.youtube.com/watch?v=hFTqQ4boR-s'),
 ("Innovating medicine with AI: Here's what to know - YouTube",
  'https://www.youtube.com/watch?v=wAqg7FtySNk'),
 ('Neural Networks & AI Explained - YouTube',
  'https://www.youtube.com/watch?v=_ZE4B0sV7BM'),
 ('Impact of artificial intelligence: AI advancement changes businesses in unexpected ways - YouTube',
  'https://www.youtube.com/watch?v=qWxjoBgd84k'),
 ('How Infrastructure is Powering the Age of AI - YouTube',
  'https://www.youtube.com/watch?v=BCc29YKiOGk'),
 ('AI Trends for 2025 - YouTube',
  'https://www.youtube.com/watch?v=5zuF4Ys1eAw'),
 ('How China’s New AI Model DeepSeek Is Threatening U.S. Dominanc

# load the google API key 

In [34]:
# Load environment variables in a file called .env
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [35]:
# Start Google API

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
MODEL = "gemini-1.5-flash"
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
patterns = [
        r"(?:v=|/)([a-zA-Z0-9_-]{11})",  # Matches v=... or /...
        r"(?:embed/|v/|watch\?v=|youtu.be/|/v/)([a-zA-Z0-9_-]{11})", # Matches embed, v/, watch?v=, youtu.be, /v/
        r"youtu\.be/([a-zA-Z0-9_-]{11})", # Matches youtu.be short links
        r"(?:/shorts/|/live/)([a-zA-Z0-9_-]{11})", # Matches youtube shorts or live
    ]

In [36]:
# Define the youtube transcript class 
class YTVideoTranscripter:
    def __init__(self, url, languages=["en"]):
        self.url = url
        self.language = languages
        self.video_id = self.extract_video_id()
        self.transcription = self.get_transcript()
        self.systemPrompt = """
        You are a helpful assistant who provides concise and accurate summaries of text. Your task is to:
        
        - Capture the key points of the content.
        - Keep the summary brief and easy to understand.
        - Avoid summarizing overly lengthy texts or breaking them into excessively short summaries.
        - Use bullet points where appropriate to enhance clarity and structure.
        """
        
    def extract_video_id(self):
        """
        Extracts the YouTube video ID from a given URL.
        Supports both regular and shortened URLs.
        """
        # Regular expression to match YouTube video URL and extract the video ID
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, self.url)
        
        if match:
            self.match = match
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def get_transcript(self):
        try:
            # Try to get the transcript in the desired language (Indonesian by default)
            transcript = YouTubeTranscriptApi.get_transcript(self.video_id, languages=[self.language])
            # Join all the 'text' fields into a single string
            return " ".join([item['text'] for item in transcript])
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return None
            
    def transcriptSummarizer(self):
        model = genai.GenerativeModel(model_name=MODEL,system_instruction=self.systemPrompt)
        response = model.generate_content(self.transcription)
        return response.text
        
        
    def __str__(self):
        return f"Video ID: {self.video_id}"

In [38]:
# Example usage
video_url = "https://www.youtube.com/watch?v=LUttsadgSEY"
for url in ytLinks.titles:
    time.sleep(10)
    yt_summary = YTVideoTranscripter(url[1], languages="en").transcriptSummarizer()
    print(yt_summary)

InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [15]:
yt_video.transcriptSummarizer()

"This video presents five AI frameworks useful for building AI applications:\n\n*   **LangChain:** A Python-based framework for working with LLMs, simplifying tasks like prompt templates, output parsing, and managing vector databases.  It facilitates creating chains of operations.\n\n*   **LangFlow:** A visual, low-code editor similar to LangChain, allowing users to build LLM-based apps via drag-and-drop functionality.\n\n*   **Ollama:** A free, open-source tool for running LLMs locally on your computer, eliminating reliance on third-party services. Requires high-end hardware for optimal performance.\n\n*   **LlamaIndex:** A Python framework focused on handling large datasets for AI applications, offering features like document extraction and text chunking.  It emphasizes working with enterprise-grade data.\n\n*   **Hugging Face Transformers:** An open-source Python module simplifying the use of transformer-based models for various tasks (NLP, audio, video processing), offering pre-tra